In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')
DATA_DIR_PATH = "/content/drive/MyDrive/DATA/birdsDatasets/birds_525_species/"
bm_path = "/content/drive/MyDrive/Models/tf_birds_bm.keras"
train_data_dir = DATA_DIR_PATH + 'train'
val_data_dir = DATA_DIR_PATH + "valid"
test_data_dir = DATA_DIR_PATH + "test"
BATCH_SIZE = 128
IMAGE_SIZE = (228, 228)

In [3]:
# tpu intialization
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)

In [4]:
def train():
    # base_model = tf.keras.applications.ResNet50(weights = "imagenet", include_top = False,
    #                                             input_shape = (228, 228, 3))
    base_model = MobileNetV2(weights= "imagenet", include_top = False, input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model.trainable = False # do not train the weights of the model
    model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation = "relu"),
        tf.keras.layers.Dense(525, activation = "softmax")
    ])
    model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])
    return model

def trainModel(tpu = False):
    if tpu:
        with strategy.scope():
            return train()
    else:
        return train()

In [5]:
train_datagenerator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
)

validation_datagenerator = ImageDataGenerator(rescale = 1./255)

test_datagenerator = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagenerator.flow_from_directory(
    train_data_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)

validation_generator = validation_datagenerator.flow_from_directory(
    val_data_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)

test_generator = test_datagenerator.flow_from_directory(
    test_data_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

Found 84635 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.


In [7]:
def preprocess_image(image, label):
    # image = tf.io.read_file(image)
    # image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = image/255.0
    return image, label

train_dataset = train_dataset.map(preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)
validation_dataset = validation_dataset.map(preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)

steps_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
validation_steps = tf.data.experimental.cardinality(validation_dataset).numpy()

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = bm_path,
    monitor= "val_accuracy",
    save_best_only = True,
    mode  = "max",
    verbose = True
)

model = trainModel(tpu = False)

history = model.fit(train_dataset,
            validation_data = validation_dataset,
            steps_per_epoch = steps_per_epoch,
            validation_steps = validation_steps,
            epochs = 10,
            callbacks = checkpoint_callback
            )

In [25]:
import pandas as pd 
CSV_MODEL_INFO_FILE = "/media/kali/system2/home/program/AI_ML/AI_ML_DEV_PY/BirdsClassification/birds.csv"

def getLabelIdDict():
    label_id_dict = {}
    df = pd.read_csv(CSV_MODEL_INFO_FILE)
    labels = sorted(df["labels"].value_counts().keys())
    class_ids = sorted(df["class id"].value_counts().keys())
    for label, class_id in zip(labels, class_ids):
        label_id_dict[class_id] = label
    return label_id_dict

getLabelIdDict()[0.0]

'ABBOTTS BABBLER'